<a href="https://colab.research.google.com/github/RuTiO2le/SabiSAM/blob/f/AdaptiveSAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Nov 12 14:20:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/RuTiO2le/SabiSAM.git
!mv SabiSAM biastuning
%cd biastuning

Cloning into 'SabiSAM'...
remote: Enumerating objects: 2945, done.
remote: Total 2945 (delta 0), reused 0 (delta 0), pack-reused 2945
Receiving objects: 100% (2945/2945), 322.89 MiB | 12.60 MiB/s, done.
Resolving deltas: 100% (311/311), done.
Updating files: 100% (2721/2721), done.
/content/biastuning


In [ ]:
!pip install -r requirements.txt > installment_log.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
bigframes 0.13.0 requires scikit-learn>=1.2.2, but you have scikit-learn 1.2.0 which is incompatible.
google-colab 1.0.0 requires google-auth==2.17.3, but you have google-auth 2.15.0 which is incompatible.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 1.5.2 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.28.2 which is incompatible.
tensorboard 2.14.1 requires google-au

In [ ]:
!rm /content/biastuning/Sabi/512x512/Train/mask_512/*.png

In [ ]:
# root_path = '/content/biastuning/Sabi/512x512/Train'

# data_config = f"""data_transforms:
#   a_min: 0
#   a_max: 512
#   img_size: 512
#   use_random_crop: False
#   use_rotation: True
#   rotation_angle: 10
#   use_saturation: False
#   saturation: 2
#   use_brightness: False
#   brightness: 2
#   use_horizontal_flip: False
# data:
#   name: CHOLEC 8K
#   root_path: '{root_path}'
#   label_list: [1,2,3]
#   label_names: ['Fair', 'Poor', 'Severe']
#   volume_channel: 2"""

# with open("config_cholec8k.yml", "w") as f:
#     f.write(data_config)

In [ ]:
# 学習
!python driver_scratchpad.py --model_config model_biastuning.yml --data_config config_cholec8k.yml --save_path "./temp.pth"

{'USE_TEXT_PROMPT': True, 'USE_IMAGE_PROMPT': False, 'LOCATION': 'prepend', 'DROPOUT': 0, 'NUM_TOKENS': 5}
100%|███████████████████████████████████████| 354M/354M [00:06<00:00, 56.3MiB/s]
number of trainable parameters:  5103920
Training parameters: 
----------
batch size:  32
num epochs:  1000
Epoch 0/999
----------
Traceback (most recent call last):
  File "/content/biastuning/driver_scratchpad.py", line 208, in <module>
    main_train(data_config, model_config, args.pretrained_path, args.save_path, args.training_strategy, device=args.device)
  File "/content/biastuning/driver_scratchpad.py", line 182, in main_train
    model = train_dl(model, dataloader_dict, dataset_sizes, criterion, optimizer, exp_lr_scheduler, save_path, num_epochs=training_params['num_epochs'], bs=training_params['batch_size'], device=device)
  File "/content/biastuning/train.py", line 147, in train_dl
    outputs = model(inputs, text)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", 

In [ ]:
%cd eval/endovis
!export IMG_PATH="/content/biastuning/Sabi/512x512/Test/img"
!export GT_PATH="/content/biastuning/Sabi/512x512/Test/mask"
!export PATH_MODEL="/content/biastuning/temp.pth"

In [ ]:
import os
from PIL import Image

base = '/content/biastuning/Sabi/512x512/Test'

# マスク画像をjpegに
mask_dir_name = os.path.join(base, "mask_512")
files = os.listdir(mask_dir_name)
number_of_exist_masks = []

for f in files:
    im = Image.open(os.path.join(mask_dir_name, f))
    im = im.convert("RGB")
    im.save(os.path.join(mask_dir_name, f"{f[:-4]}.jpeg"))
    number_of_exist_masks.append(int(f[:-4]))

In [ ]:
!rm /content/biastuning/Sabi/512x512/Test/mask_512/*.png

In [ ]:
endvis = """data_transforms:
  a_min: 0
  a_max: 255
  img_size: 256
  use_random_crop: False
  use_rotation: False
  rotation_angle: 10
  use_saturation: False
  saturation: 2
  use_brightness: False
  brightness: 2
  use_horizontal_flip: False
data:
  name: ENDOVIS
  root_path: '/media/ubuntu/New Volume/jay/endovis17/instrument_1_4_testing/'
  label_list: [1,2,3,4,5,6,7,8,9]
  label_names: ['Left Prograsp Forceps', 'Maryland Bipolar Forceps', 'Right Prograsp Forceps', 'Left Large Needle Driver', 'Right Large Needle Driver', 'Left Grasping Retractor', 'Right Grasping Retractor', 'Vessel Sealer', 'Monopolar Curved Scissors']
  volume_channel: 2"""

with open("eval/endovis/config_endovis_test.yml", "w") as f:
    f.write(endvis)

In [ ]:
# 推論
python generate_predictions.py --model_config config_model_test.yml --data_config config_endovis_test.yml --data_folder $IMG_PATH --gt_path $GT_PATH --save_path "./temp_results" --pretrained_path $PATH_MODEL

## git整備する前に使用していた環境構築

In [ ]:
!git clone https://github.com/JayParanjape/biastuning.git

Cloning into 'biastuning'...
remote: Enumerating objects: 337, done.
remote: Counting objects: 100% (337/337), done.
remote: Compressing objects: 100% (206/206), done.
remote: Total 337 (delta 206), reused 249 (delta 123), pack-reused 0
Receiving objects: 100% (337/337), 3.89 MiB | 7.51 MiB/s, done.
Resolving deltas: 100% (206/206), done.


In [ ]:
%cd biastuning

/content/biastuning


In [ ]:
!unzip /content/drive/MyDrive/datasets/Corrosion_Condition_State_Classification.zip -qq

Archive:  /content/drive/MyDrive/datasets/Corrosion_Condition_State_Classification.zip
caution: filename not matched:  -qq


In [ ]:
import yaml
import re

# バージョン指定の有無
version = True
# yamlファイル
yaml_path = "biastuning_env.yml"

with open(yaml_path) as data:
    yaml_obj = yaml.safe_load(data)

    requirements = []
    for dep in yaml_obj['dependencies']:
        if isinstance(dep, str):
            dep_l = re.split('=', dep)
            # 除外対象
            res = re.match('python|pip|setuptools', dep)
            if res is None:
                if version and len(dep_l) == 2:
                    requirements.append(dep_l[0] + '==' + dep_l[1])
                else:
                    requirements.append(dep_l[0])
        else:
            for preq in dep.get('pip', []):
                preq_s = re.sub('>=|<=|>|<|==', '#', preq)
                preq_s_l = re.split('#', preq_s)

                if preq_s_l[0]:
                    res = re.match('-e', preq_s_l[0])
                    if res is None:
                        new_string = preq_s_l[0]
                    else:
                        new_string = preq.lstrip("-e | -e .")
                        new_string = new_string.strip()

                if version:
                    new_string = preq.lstrip("-e | -e .")
                    requirements.append(new_string)
                else:
                    requirements.append(new_string)

with open('requirements.txt', 'w') as fp:
    for requirement in requirements:
        print(requirement, file=fp)

FileNotFoundError: ignored

In [ ]:
req = """absl-py==1.3.0
addict==2.4.0
appdirs==1.4.4
argparse==1.4.0
batchgenerators==0.25
beautifulsoup4==4.11.1
cachetools==5.2.0
chardet==3.0.4
charset-normalizer==3.1.0
click==8.1.3
cmake==3.26.3
contourpy==1.0.7
cycler==0.11.0
docker-pycreds==0.4.0
efficientnet-pytorch==0.7.1
entrypoints==0.3
exceptiongroup==1.1.1
filelock==3.8.2
flake8==3.7.9
fonttools==4.39.3
ftfy==6.1.1
future==0.18.2
gdown==4.6.0
gensim==4.3.1
gitdb==4.0.10
gitpython==3.1.31
google-auth==2.15.0
google-auth-oauthlib==0.4.6
googletrans==3.0.0
grpcio==1.51.1
h11==0.9.0
h2==3.2.0
h5py==3.8.0
hpack==3.0.0
hstspreload==2023.1.1
httpcore==0.9.1
httpx==0.13.3
huggingface-hub==0.11.1
hyperframe==5.2.0
idna==2.10
imageio==2.28.0
importlib-metadata==5.2.0
importlib-resources==5.12.0
iniconfig==2.0.0
isort==4.3.21
jinja2==3.1.2
joblib==1.2.0
kiwisolver==1.4.4
lazy-loader==0.2
linecache2==1.0.0
lit==16.0.3
littleutils==0.2.2
markdown==3.4.1
markupsafe==2.1.1
matplotlib==3.7.1
mccabe==0.6.1
mpmath==1.3.0
munch==3.0.0
networkx==3.1
nibabel==5.1.0
nltk==3.8.1
numpy==1.24.2
nvidia-cublas-cu11==11.10.3.66
nvidia-cuda-cupti-cu11==11.7.101
nvidia-cuda-nvrtc-cu11==11.7.99
nvidia-cuda-runtime-cu11==11.7.99
nvidia-cudnn-cu11==8.5.0.96
nvidia-cufft-cu11==10.9.0.58
nvidia-curand-cu11==10.2.10.91
nvidia-cusolver-cu11==11.4.0.1
nvidia-cusparse-cu11==11.7.4.91
nvidia-nccl-cu11==2.14.3
nvidia-nvtx-cu11==11.7.91
oauthlib==3.2.2
ogb==1.3.5
opencv-python==4.6.0.66
outdated==0.2.2
packaging==22.0
pandas==1.5.2
pathtools==0.1.2
pillow==9.5.0
pluggy==1.0.0
pretrainedmodels==0.7.4
protobuf==3.20.3
psutil==5.9.4
pyasn1==0.4.8
pyasn1-modules==0.2.8
pycocotools==2.0.6
pycodestyle==2.5.0
pyflakes==2.1.1
pyparsing==3.0.9
pytest==7.3.1
pytz==2022.7
pywavelets==1.4.1
pyyaml==6.0
regex==2022.10.31
requests==2.28.2
requests-oauthlib==1.3.1
rfc3986==1.5.0
rsa==4.9
scikit-image==0.20.0
scikit-learn==1.2.0
scipy==1.9.1
sentry-sdk==1.18.0
setproctitle==1.3.2
simpleitk==2.2.1
smart-open==6.3.0
smmap==5.0.0
sniffio==1.3.0
soupsieve==2.3.2.post1
supervision==0.3.2
surface-distance-based-measures==0.1
sympy==1.12
tabulate==0.9.0
tb-nightly==2.12.0a20221225
tensorboard-data-server==0.6.1
tensorboard-plugin-wit==1.8.1
textaugment==1.3.4
textblob==0.17.1
threadpoolctl==3.1.0
tifffile==2023.4.12
timm==0.6.13
tokenizers==0.13.3
tomli==2.0.1
torch==2.0.1
torchaudio
torchvision==0.15.2
tqdm==4.64.1
traceback2==1.4.0
transformers==4.27.4
triton==2.0.0
unittest2==1.1.0
urllib3==1.26.15
wandb==0.14.0
werkzeug==2.2.2
wget==3.2
wheel==0.38.4
wilds==1.2.2
yacs==0.1.8
yapf==0.29.0"""

with open('requirements.txt', 'w') as fp:
    print(req, file=fp)

In [ ]:
root_path = '/content/drive/MyDrive/datasets/corrosion_condition_state_classification/512x512/Test'

data_config = f"""data_transforms:
  a_min: 0
  a_max: 512
  img_size: 512
  use_random_crop: False
  use_rotation: True
  rotation_angle: 10
  use_saturation: False
  saturation: 2
  use_brightness: False
  brightness: 2
  use_horizontal_flip: False
data:
  name: CHOLEC 8K
  root_path: '{root_path}'
  label_list: [1,2,3]
  label_names: ['Fair', 'Poor', 'Severe']
  volume_channel: 2"""

with open("config_cholec8k.yml", "w") as f:
    f.write(data_config)

In [ ]:
root_path = '/content/biastuning/Sabi/512x512/Train'

data_config = f"""data_transforms:
  a_min: 0
  a_max: 512
  img_size: 512
  use_random_crop: False
  use_rotation: True
  rotation_angle: 10
  use_saturation: False
  saturation: 2
  use_brightness: False
  brightness: 2
  use_horizontal_flip: False
data:
  name: CHOLEC 8K
  root_path: '{root_path}'
  label_list: [1,2,3]
  label_names: ['Fair', 'Poor', 'Severe']
  volume_channel: 2"""

with open("config_cholec8k.yml", "w") as f:
    f.write(data_config)

In [ ]:
!python driver_scratchpad.py --model_config model_biastuning.yml --data_config config_cholec8k.yml --save_path "./temp.pth"

In [ ]:
# これだとエラー（これに関しては上も共通）
# !python driver_scratchpad.py --model_config model_biastuning.yml --data_config config_cholec8k.yml --save_path --pretrained_path "/content/drive/MyDrive/endovis17/ev17_textaffine_decoder_biastuning_blanklabels_focal.pth"

requirementsを書き換えない場合

In [ ]:
# !pip install ftfy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.9 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/openai/CLIP.git

Cloning into 'CLIP'...
remote: Enumerating objects: 251, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 251 (delta 3), reused 3 (delta 0), pack-reused 243
Receiving objects: 100% (251/251), 8.93 MiB | 11.84 MiB/s, done.
Resolving deltas: 100% (127/127), done.


In [ ]:
!mv /content/biastuning/CLIP/clip/bpe_simple_vocab_16e6.txt.gz /content/biastuning/clip/bpe_simple_vocab_16e6.txt.gz

In [ ]:
ls /content/drive/MyDrive/datasets/corrosion_condition_state_classification/512x512/Test

images_512/  mask_512/


data_utils.pyのL480を以下のように変更  
```python
    def populate_lists(self):
        path2 = os.path.join(self.root_path, 'mask_512')
        for folder in (self.folder_list):
            path1 = os.path.join(self.root_path, 'images_512')
            for im in sorted(os.listdir(path1)):
                im_name = im[:-5]
                im_path = os.path.join(path1, im)
                label_img_path = os.path.join(path2, im_name+'.jpeg')
                self.img_names.append(im_name)
                self.img_path_list.append(os.path.join(im_path))
                self.label_path_list.append(os.path.join(label_img_path))
                self.label_list.append('')
```

data_utils.pyの
`no_text_mode=False`を全て`no_text_mode=True`に

- clip.pyはもう一回実行すれば解消

model.pyのL87
```python
/content/drive/MyDrive/endovis17/ev17_textaffine_decoder_biastuning_blanklabels_focal.pth
```

## condaをインストールしようとした残骸

In [ ]:
!echo $PYTHONPATH

/env/python


In [ ]:
%env PYTHONPATH=

env: PYTHONPATH=


In [ ]:
%%bash

MINICONDA_INSTALLER_SCRIPT=Miniconda3-latest-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
!which conda

/usr/local/bin/conda


In [ ]:
ls

axialnet.py           config_tmp.yml          model.py
baselines.py          config_ultrasound.yml   prompt_adapted_segment_anything/
biastuning_env.yml    data_transforms/        README.md
clip/                 data_utils.py           scratchpad.ipynb
config_chestxdet.yml  driver_scratchpad.py    test.py
config_cholec8k.yml   eval/                   train_baselines.py
config_endovis18.yml  model_baseline.yml      train.py
config_endovis.yml    model_biastuning.yml    utils.py
config_idrid.yml      model_components/       vit_seg_configs.py
config_kvasirseg.yml  model_fdn.yml           vit_seg_modeling.py
config_lits.yml       model_prompttuning.yml  vit_seg_modeling_resnet_skip.py


In [ ]:
%cd biastuning

[Errno 2] No such file or directory: 'biastuning'
/content/biastuning
Solving environment: | failed

ResolvePackageNotFound: 
  - ml-collections

usage: conda [-h] [-v] [--no-plugins] [-V] COMMAND ...
conda: error: argument COMMAND: invalid choice: 'activate' (choose from 'clean', 'compare', 'config', 'create', 'info', 'init', 'install', 'list', 'notices', 'package', 'remove', 'uninstall', 'rename', 'run', 'search', 'update', 'upgrade', 'content-trust', 'doctor', 'repoquery', 'env')


In [ ]:
!conda init
!conda env create --file=biastuning_env.yml
!conda activate biastuning_env

no change     /usr/local/condabin/conda
no change     /usr/local/bin/conda
no change     /usr/local/bin/conda-env
no change     /usr/local/bin/activate
no change     /usr/local/bin/deactivate
no change     /usr/local/etc/profile.d/conda.sh
no change     /usr/local/etc/fish/conf.d/conda.fish
no change     /usr/local/shell/condabin/Conda.psm1
no change     /usr/local/shell/condabin/conda-hook.ps1
no change     /usr/local/lib/python3.11/site-packages/xontrib/conda.xsh
no change     /usr/local/etc/profile.d/conda.csh
modified      /root/.bashrc

==> For changes to take effect, close and re-open your current shell. <==

Solving environment: | failed

ResolvePackageNotFound: 
  - ml-collections

usage: conda [-h] [-v] [--no-plugins] [-V] COMMAND ...
conda: error: argument COMMAND: invalid choice: 'activate' (choose from 'clean', 'compare', 'config', 'create', 'info', 'init', 'install', 'list', 'notices', 'package', 'remove', 'uninstall', 'rename', 'run', 'search', 'update', 'upgrade', 'con